In [1]:
import cv2                 # working with, mainly resizing, images
import numpy as np         # dealing with arrays
import os                  # dealing with directories
from random import shuffle # mixing up or currently ordered data that might lead our network astray in training.
from tqdm import tqdm      # a nice pretty percentage bar for tasks. Thanks to viewer Daniel BA1/4hler for this suggestion

import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

TRAIN_DIR = './data/alternate_data/train'
TEST_DIR = './data/test'
IMG_SIZE = 50
LR = 1e-3

MODEL_NAME = 'dogsvscats-{}-{}.model'.format(LR, '2conv-basic') # just so we remember which saved model is which, sizes must match
print(MODEL_NAME)

/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


dogsvscats-0.001-2conv-basic.model


In [2]:
def onehot_label_img(img):
    word_label = img.split('.')[-3]
    # conversion to one-hot array [cat,dog]
    #                            [much cat, no dog]
    if word_label == 'cat': return [1,0]
    #                             [no cat, very doggo]
    elif word_label == 'dog': return [0,1]

In [3]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = onehot_label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data

def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR,img)
        img_num = img.split('.')[0]
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img), img_num])
        
    shuffle(testing_data)
    np.save('test_data.npy', testing_data)
    return testing_data

In [4]:
train_data = create_train_data()

100%|██████████| 25000/25000 [00:51<00:00, 484.50it/s]


In [5]:
# ConvNet
convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')
print(convnet.shape)
convnet = conv_2d(convnet, 32, 5, activation='relu')
print(convnet.shape)
convnet = max_pool_2d(convnet, 5)
print(convnet.shape)
convnet = conv_2d(convnet, 64, 5, activation='relu')
print(convnet.shape)
convnet = max_pool_2d(convnet, 5)
print(convnet.shape)

convnet = conv_2d(convnet, 128, 5, activation='relu')
print(convnet.shape)
convnet = max_pool_2d(convnet, 5)
print(convnet.shape)
convnet = conv_2d(convnet, 64, 5, activation='relu')
print(convnet.shape)
convnet = max_pool_2d(convnet, 5)
print(convnet.shape)

convnet = conv_2d(convnet, 32, 5, activation='relu')
print(convnet.shape)
convnet = max_pool_2d(convnet, 5)
print(convnet.shape)

convnet = fully_connected(convnet, 1024, activation='relu')
print(convnet.shape)
convnet = dropout(convnet, 0.8)
print(convnet.shape)

convnet = fully_connected(convnet, 2, activation='softmax')
print(convnet.shape)
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')
print(convnet.shape)
model = tflearn.DNN(convnet, tensorboard_dir='log')

if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')
    
train = train_data[:-500]
test = train_data[-500:]

X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1] for i in test]
model.fit({'input': X}, {'targets': Y}, n_epoch=7, validation_set=({'input': test_x}, {'targets': test_y}), snapshot_step=500, show_metric=True)

Training Step: 2680  | total loss: 0.38540 | time: 72.931s
| Adam | epoch: 007 | loss: 0.38540 - acc: 0.8236 -- iter: 24448/24500
Training Step: 2681  | total loss: 0.38068 | time: 74.114s
| Adam | epoch: 007 | loss: 0.38068 - acc: 0.8257 | val_loss: 0.46552 - val_acc: 0.7840 -- iter: 24500/24500
--
